In [1]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict
from langchain_core.runnables import RunnableLambda
import gradio as gr

# LLM
llm = ChatOpenAI(model="gpt-4o")

# Estado
class State(TypedDict):
    task: str
    lang: str         # "es" or "en"
    profession: str   # "Developer", "Lawyer", etc.
    sector: str       # "AI", "Finance", "Photography"
    news: str
    meaning: str
    action: str
    linkedin_post: str
    poc_ideas: str
    compounding: str
    final_summary: str

# Helper para idioma
def lang_prefix(lang: str, es: str, en: str):
    return es if lang == "es" else en

# Nodo: Noticias
def news_agent(state: State) -> State:
    prompt = f"""{lang_prefix(state['lang'],
    "Sos un analista experto. Extraé 3–5 noticias recientes relacionadas a la profesión y sector del usuario. Redactá en lenguaje claro y útil, sin inventar.",
    "You are an expert analyst. Extract 3–5 recent news items related to the user's profession and sector. Write clearly and practically, no fabrication.")}

Profession: {state['profession']}
Sector: {state['sector']}
Task focus: {state['task']}
"""
    result = llm.invoke(prompt)
    state["news"] = result.content
    return state

# Nodo: Significado
def meaning_agent(state: State) -> State:
    prompt = f"""{lang_prefix(state['lang'],
    "Sos un coach de carrera. Explicá cómo cada noticia representa una oportunidad real para la profesión y sector del usuario.",
    "You are a career coach. Explain how each news translates into real opportunities for the user's profession and sector.")}

Profession: {state['profession']}
Sector: {state['sector']}
News:
{state['news']}
"""
    result = llm.invoke(prompt)
    state["meaning"] = result.content
    return state

# Nodo: Acción diaria
def action_agent(state: State) -> State:
    prompt = f"""{lang_prefix(state['lang'],
    "Proponé una micro-acción diaria (≤15 min) para {state['profession']} en el sector {state['sector']}. Debe ser concreta, ejecutable hoy y visible.",
    "Suggest one micro-action (≤15 min) for a {state['profession']} in {state['sector']}. Must be concrete, executable today, and visible.")}

News: {state['news']}
Meaning: {state['meaning']}
"""
    result = llm.invoke(prompt)
    state["action"] = result.content
    return state

# Nodo: LinkedIn Post
def linkedin_agent(state: State) -> State:
    prompt = f"""Generate 2 LinkedIn posts ({lang_prefix(state['lang'], "uno en español y otro en inglés", "one in English and one in Spanish")}),
style: authoritative, inspiring, not egocentric. Goal: attract inbound high-value leads.

Profession: {state['profession']}
Sector: {state['sector']}
Context:
{state['news']}
Meaning:
{state['meaning']}
Daily Action:
{state['action']}
"""
    result = llm.invoke(prompt)
    state["linkedin_post"] = result.content
    return state

# Nodo: POCs
def poc_agent(state: State) -> State:
    prompt = f"""{lang_prefix(state['lang'],
    "Generá 3 mini-POCs (≤45 min) adaptados a la profesión {state['profession']} en {state['sector']}. Deben ser acciones que documenten progreso (ej. commit, foto, análisis legal).",
    "Generate 3 mini-POCs (≤45 min) tailored for {state['profession']} in {state['sector']}. They must be visible progress actions (e.g., commit, photo, legal analysis).")}

News:
{state['news']}
"""
    result = llm.invoke(prompt)
    state["poc_ideas"] = result.content
    return state

# Nodo: Compounding
def compounding_agent(state: State) -> State:
    prompt = f"""{lang_prefix(state['lang'],
    "Explicá cómo la acción diaria, los posts y los POCs se acumulan estratégicamente para ser top en su profesión dentro del sector {state['sector']}.",
    "Explain how the daily action, LinkedIn posts, and POCs strategically compound to make the user top in {state['profession']} within {state['sector']}.")}

Action:
{state['action']}
Posts:
{state['linkedin_post']}
POCs:
{state['poc_ideas']}
"""
    result = llm.invoke(prompt)
    state["compounding"] = result.content
    return state

# Nodo: Resumen final
def final_summary(state: State) -> State:
    prompt = f"""{lang_prefix(state['lang'],
    "📋 Resumen Final de la Lectura de Hoy",
    "📋 Final Summary of Today's Reading")}

📰 Noticias:
{state['news']}

💡 Oportunidades:
{state['meaning']}

⚡ Acción diaria:
{state['action']}

🔗 LinkedIn:
{state['linkedin_post']}

🛠️ POCs:
{state['poc_ideas']}

📈 Compounding:
{state['compounding']}
"""
    result = llm.invoke(prompt)
    state["final_summary"] = result.content
    return state

# Construcción del grafo
builder = StateGraph(State)
builder.add_node("News", RunnableLambda(news_agent))
builder.add_node("Meaning", RunnableLambda(meaning_agent))
builder.add_node("Action", RunnableLambda(action_agent))
builder.add_node("LinkedIn", RunnableLambda(linkedin_agent))
builder.add_node("POCs", RunnableLambda(poc_agent))
builder.add_node("Compounding", RunnableLambda(compounding_agent))
builder.add_node("Final", RunnableLambda(final_summary))

builder.set_entry_point("News")
builder.add_edge("News", "Meaning")
builder.add_edge("Meaning", "Action")
builder.add_edge("Action", "LinkedIn")
builder.add_edge("LinkedIn", "POCs")
builder.add_edge("POCs", "Compounding")
builder.add_edge("Compounding", "Final")
builder.add_edge("Final", END)

graph = builder.compile(checkpointer=MemorySaver())

# Interfaz Gradio (para demo rápida)
def run_pipeline(task, lang, profession, sector):
    result = graph.invoke(
        {"task": task, "lang": lang, "profession": profession, "sector": sector},
        config={"configurable": {"thread_id": "gradio-run"}}
    )
    return (
        result["news"],
        result["meaning"],
        result["action"],
        result["linkedin_post"],
        result["poc_ideas"],
        result["compounding"],
        result["final_summary"]
    )

with gr.Blocks() as demo:
    gr.Markdown("## 🌍 Daily Reader MVP (LangChain + Gradio)")
    task_input = gr.Textbox(label="📌 Extra details (focus)", lines=2)
    profession_input = gr.Textbox(label="👩‍💻 Profession", value="Developer")
    sector_input = gr.Textbox(label="🏢 Sector", value="AI")
    lang_input = gr.Radio(["es", "en"], label="🌐 Language", value="es")
    run_btn = gr.Button("🚀 Generate Daily Reading")

    with gr.Row():
        with gr.Column():
            news_out = gr.Textbox(label="📰 News", lines=8)
            meaning_out = gr.Textbox(label="💡 Opportunities", lines=8)
            action_out = gr.Textbox(label="⚡ Daily Action", lines=4)
        with gr.Column():
            linkedin_out = gr.Textbox(label="🔗 LinkedIn Posts", lines=10)
            poc_out = gr.Textbox(label="🛠️ POC Ideas", lines=8)
            comp_out = gr.Textbox(label="📈 Compounding", lines=6)
            final_out = gr.Textbox(label="📜 Final Summary", lines=12)

    run_btn.click(fn=run_pipeline, inputs=[task_input, lang_input, profession_input, sector_input], outputs=[
        news_out, meaning_out, action_out, linkedin_out, poc_out, comp_out, final_out
    ])

demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://ca8c189cb52ea82330.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
